In [1]:
"""
Problem Statement:
For a given query dates for a stock having inputs containing multiple metrics such as opening price (Open), 
highest price the stock traded at (High), how many stocks were traded (Volume) and closing price adjusted for stock splits and 
dividends (Adjusted Close); your system only needs to predict the Adjusted Close price.

In order to solve the issue, I have created a stock price predictor which builds a Long Short-Term Memory(LSTM) model based on 
Adjusted Close for a certain period and predict the price for the next 'n' number of days.

In this notebook, I have calculated the predictions for each of the stocks listed in Nifty50 and storing them 
in database named after that stock.

Note: Since I have used Colab notebook, I had to zip the folder and download the zipped file in 'models/'
directory for further processing. Please do so as this folder(unzipped version) is required for next steps.
"""

"\nProblem Statement:\nFor a given query dates for a stock having inputs containing multiple metrics such as opening price (Open), \nhighest price the stock traded at (High), how many stocks were traded (Volume) and closing price adjusted for stock splits and \ndividends (Adjusted Close); your system only needs to predict the Adjusted Close price.\n\nIn order to solve the issue, I have created a stock price predictor which builds a Long Short-Term Memory(LSTM) model based on \nAdjusted Close for a certain period and predict the price for the next 'n' number of days.\n\nIn this notebook, I have calculated the predictions for each of the stocks listed in Nifty50 and storing them \nin database named after that stock.\n\nNote: Since I have used Colab notebook, I had to zip the folder and download the zipped file in 'models/'\ndirectory for further processing. Please do so as this folder(unzipped version) is required for next steps.\n"

In [2]:
# Importing Libraries

import math
from datetime import date, timedelta
import pandas_datareader as web
import numpy as np
import pandas as pd
from sqlalchemy import create_engine

from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM

from google.colab import files

In [3]:
def get_dates():
    """
    I will be considering dates of stock ranging from current date(today) to 600 days from current day for analysis.
    Args: None
    Returns:
    today (date): Today's date
    prev_600 (date): Previous 600 day date
    """
    today = date.today()
    prev_600 = (today - timedelta(600)).strftime("%Y-%m-%d")
    return prev_600, today

In [4]:
def process_predict_generate_data(stock):
  """
  Prediction for stock are based on the following steps:-
   - Creating the dataframe by using YAHOO as datasource, today and prev_600 dates
   - Scale the dataframe using MinMaxScaler and Split it into train and test dataset in ratio of 4:1.
   - Defining the model. I have used LSTM for this task, which is a variation of Recurrent Neural Network.
   - Fit the model by using train dataset
   - Predict the stock prices based on test dataset
   - Finally store the Date, Actual Price and Prediction price in the database named after Stock
   
   Args:
   stock (string): Name of the stock
   Returns:
   None; Creates a database containing prediction price
  """
  prev_600, today = get_dates()
  df = web.DataReader(stock+'.NS', data_source='yahoo', start=prev_600, end=today)
  data = df.filter(['Adj Close']) # prediction for the Adjusted Close price
  dataset = data.values
  training_data_len = math.ceil(len(dataset)*.8)

  # Scaling the data
  scaler = MinMaxScaler(feature_range=(0,1))
  scaled_data = scaler.fit_transform(dataset)

  # Creating training dataset
  train_data = scaled_data[0:training_data_len, : ]
  x_train = []
  y_train = []
  for i in range(60, len(train_data)):
    x_train.append(train_data[i-60:i, 0])
    y_train.append(train_data[i, 0])

  x_train, y_train = np.array(x_train), np.array(y_train)
  x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

  # Building LSTM model
  model = Sequential()
  model.add(LSTM(50, return_sequences=True, input_shape=(x_train.shape[1], 1)))
  model.add(LSTM(50,return_sequences=False))
  model.add(Dense(25))
  model.add(Dense(1))
  model.compile(optimizer='adam', loss='mean_squared_error')
  model.fit(x_train, y_train, batch_size=1, epochs=10, verbose=1)

  # Creating testing dataset
  test_data = scaled_data[training_data_len - 60: , :]
  x_test = []
  y_test = dataset[training_data_len:, :]
  for i in range(60, len(test_data)):
    x_test.append(test_data[i-60:i, 0])
  x_test = np.array(x_test)
  x_test = np.reshape(x_test, (x_test.shape[0],x_test.shape[1], 1))

  #Get the predictions
  predictions = model.predict(x_test)
  predictions = scaler.inverse_transform(predictions)

  train = data[: training_data_len]
  valid = data[training_data_len: ]
  valid['Predictions'] = predictions
  valid['Date'] = valid.index
  database_filename = stock+'.db'
  engine = create_engine('sqlite:///{}'.format(database_filename))

  table_name = database_filename.split('.')[0]
  valid.to_sql(table_name, engine, index=False, if_exists = 'replace')

In [5]:
def generate_database():
  """
  The below listed stocks are in NIFTY50. For each of these stocks, processing and predictions are made and stored in the database
  Args: None
  Returns:
  None; Creates a database containing prediction price
  """
  stock_data = [
  "ADANIPORTS","ASIANPAINT","AXISBANK","BAJAJ-AUTO",
  "BAJFINANCE","BAJAJFINSV","BPCL","BHARTIARTL","BRITANNIA",
  "CIPLA","COALINDIA","DIVISLAB","DRREDDY","EICHERMOT",
  "GAIL","GRASIM","HCLTECH","HDFCBANK","HDFCLIFE","HEROMOTOCO",
  "HINDALCO","HINDUNILVR","HDFC","ICICIBANK","ITC","IOC","INDUSINDBK",
  "INFY","JSWSTEEL","KOTAKBANK","LT","M&M","MARUTI","NTPC","NESTLEIND",
  "ONGC","POWERGRID","RELIANCE","SBILIFE","SHREECEM","SBIN","SUNPHARMA","TCS",
  "TATAMOTORS","TATASTEEL","TECHM","TITAN","UPL","ULTRACEMCO","WIPRO"
  ]
  for stock in stock_data:
    process_predict_generate_data(stock)


# Function call to generate the database for all thestocks in 'stock_data' variable
generate_database()

"""
Below lines are used to zip all the generated *.db to 'Database.Stock.zip'
This is followed by downloading the file. The reason why I have done this is because I need to store
them(unzipped version) in my 'models/' directory for further processing.
"""
!zip -r DatabaseStock.zip *.db
files.download("DatabaseStock.zip")

Epoch 1/10
264/264 [==============================] - 9s 23ms/step - loss: 0.0061
Epoch 2/10
264/264 [==============================] - 6s 22ms/step - loss: 8.6992e-04
Epoch 3/10
264/264 [==============================] - 6s 22ms/step - loss: 8.7577e-04
Epoch 4/10
264/264 [==============================] - 6s 22ms/step - loss: 6.7849e-04
Epoch 5/10
264/264 [==============================] - 6s 22ms/step - loss: 6.4267e-04
Epoch 6/10
264/264 [==============================] - 6s 22ms/step - loss: 5.5250e-04
Epoch 7/10
264/264 [==============================] - 6s 22ms/step - loss: 5.4603e-04
Epoch 8/10
264/264 [==============================] - 6s 23ms/step - loss: 4.3764e-04
Epoch 9/10
264/264 [==============================] - 6s 22ms/step - loss: 4.9746e-04
Epoch 10/10
264/264 [==============================] - 6s 22ms/step - loss: 3.3324e-04


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Epoch 1/10
264/264 [==============================] - 10s 25ms/step - loss: 0.0112
Epoch 2/10
264/264 [==============================] - 6s 24ms/step - loss: 0.0023
Epoch 3/10
264/264 [==============================] - 6s 24ms/step - loss: 0.0022
Epoch 4/10
264/264 [==============================] - 6s 23ms/step - loss: 0.0016
Epoch 5/10
264/264 [==============================] - 6s 23ms/step - loss: 0.0012
Epoch 6/10
264/264 [==============================] - 6s 23ms/step - loss: 0.0012
Epoch 7/10
264/264 [==============================] - 6s 23ms/step - loss: 9.7324e-04
Epoch 8/10
264/264 [==============================] - 6s 23ms/step - loss: 0.0010
Epoch 9/10
264/264 [==============================] - 6s 23ms/step - loss: 8.1580e-04
Epoch 10/10
264/264 [==============================] - 6s 23ms/step - loss: 8.6855e-04


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Epoch 1/10
264/264 [==============================] - 9s 25ms/step - loss: 0.0384
Epoch 2/10
264/264 [==============================] - 6s 24ms/step - loss: 0.0052
Epoch 3/10
264/264 [==============================] - 6s 24ms/step - loss: 0.0043
Epoch 4/10
264/264 [==============================] - 6s 24ms/step - loss: 0.0038
Epoch 5/10
264/264 [==============================] - 6s 24ms/step - loss: 0.0049
Epoch 6/10
264/264 [==============================] - 7s 25ms/step - loss: 0.0045
Epoch 7/10
264/264 [==============================] - 6s 23ms/step - loss: 0.0026
Epoch 8/10
264/264 [==============================] - 6s 23ms/step - loss: 0.0031
Epoch 9/10
264/264 [==============================] - 6s 23ms/step - loss: 0.0024
Epoch 10/10
264/264 [==============================] - 6s 23ms/step - loss: 0.0048


...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Epoch 1/10
264/264 [==============================] - 10s 27ms/step - loss: 0.0068
Epoch 2/10
264/264 [==============================] - 7s 25ms/step - loss: 0.0017
Epoch 3/10
264/264 [==============================] - 7s 25ms/step - loss: 0.0014
Epoch 4/10
264/264 [==============================] - 7s 25ms/step - loss: 0.0015
Epoch 5/10
264/264 [==============================] - 7s 25ms/step - loss: 0.0012
Epoch 6/10
264/264 [==============================] - 6s 24ms/step - loss: 9.8697e-04
Epoch 7/10
264/264 [==============================] - 6s 24ms/step - loss: 0.0011
Epoch 8/10
264/264 [==============================] - 6s 24ms/step - loss: 5.8038e-04
Epoch 9/10
264/264 [==============================] - 6s 24ms/step - loss: 0.0010
Epoch 10/10
264/264 [==============================] - 6s 24ms/step - loss: 5.8243e-04
  adding: ADANIPORTS.db (deflated 82%)
  adding: ASIANPAINT.db (deflated 82%)
  adding: AXISBANK.db (deflated 82%)
  adding: BAJAJ-AUTO.db (deflated 82%)
  adding: BA

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>